In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import warnings
import seaborn as sns

warnings.filterwarnings('ignore')
%matplotlib inline
#device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 모의 1: GAS MEAN TEMP 만 다루기 위한 데이터 전처리

### 예시 : 1월 데이터

In [3]:
df = pd.read_csv('C:/Users/Sejong/Desktop/한화오션/모의csv파일/general_pivot_1.csv', encoding = 'cp949')
df = df.drop([df.columns[i] for i in range(3,38,2)], axis = 1)
df['kst'] = [df['kst'][i][11:] for i in range(0,88194)]
df.rename(columns = {'癤풼s_date' : 'date'}, inplace = True)
df_gas = df.iloc[:, :8] # gas temerature data frame
df_gas_mean = pd.concat([df.iloc[:, :2], df.iloc[:,7]], axis = 1)
df_gas_mean_new = {'date' : [],
                   'kst' : [],
                   'GAS MEAN TEMP' :[]}

for i in range(0,len(df_gas_mean['date']),2): # 0, 2, 4,...
    MEAN = (df_gas_mean['ME(S) CYL EXH GAS MEAN TEMP'][i] + df_gas_mean['ME(S) CYL EXH GAS MEAN TEMP'][i+1]) / 2
    df_gas_mean_new['date'].append(df_gas_mean['date'][i])
    df_gas_mean_new['kst'].append(df_gas_mean['kst'][i])
    df_gas_mean_new['GAS MEAN TEMP'].append(MEAN)

df_gas_mean_new = pd.DataFrame(df_gas_mean_new)
df_gas_mean_new

,date,kst,GAS MEAN TEMP
0,20230101,9:00,286.9395
1,20230101,9:01,285.8550
2,20230101,9:02,286.3170
3,20230101,9:03,285.1720
4,20230101,9:04,285.0330
...,...,...,...
44092,20230131,23:55,275.0270
44093,20230131,23:56,274.8775
44094,20230131,23:57,274.7335
44095,20230131,23:58,274.6310


### 1-8월 까지의 데이터 GAS MEAN TEMP 만 뽑아서 dataframe 작성

In [4]:
for idx in range(1,9): # 1월부터 8월까지
    df = pd.read_csv('C:/Users/Sejong/Desktop/한화오션/모의csv파일/general_pivot_{}.csv'.format(idx), encoding = 'cp949')
    df = df.drop([df.columns[i] for i in range(3,38,2)], axis = 1)
    df['kst'] = [df['kst'][i][11:] for i in range(0,len(df['kst']))]
    df.rename(columns = {'癤풼s_date' : 'date'}, inplace = True)
    df_gas = df.iloc[:, :8] # gas temerature data frame
    df_gas_mean = pd.concat([df.iloc[:, :2], df.iloc[:,7]], axis = 1)
    globals()['df_gas_mean_new_' + str(idx)] = {'date' : [],
                    'kst' : [],
                    'GAS MEAN TEMP' :[]}
    
    # 새로운 DataFrame 만들기
    for i in range(0,len(df_gas_mean['date']),2): # 0, 2, 4,...

        # 똑같은 시간(09:00, 09:00) 시간의 GAS TEMP 의 평균을 구해서 하나로 합침
        MEAN = (df_gas_mean['ME(S) CYL EXH GAS MEAN TEMP'][i] + df_gas_mean['ME(S) CYL EXH GAS MEAN TEMP'][i+1]) / 2

        # df_gas_mean_new_{}인 DataFrame에 값 할당
        globals()['df_gas_mean_new_' + str(idx)]['date'].append(df_gas_mean['date'][i])
        globals()['df_gas_mean_new_' + str(idx)]['kst'].append(df_gas_mean['kst'][i])
        globals()['df_gas_mean_new_' + str(idx)]['GAS MEAN TEMP'].append(MEAN)
    
    globals()['df_gas_mean_new_' + str(idx)] = pd.DataFrame( globals()['df_gas_mean_new_' + str(idx)]) # df_gas_mewn_new_1,2...8

# 결측치 평균값으로 채우기
df_gas_mean_new_7['GAS MEAN TEMP'][13884:13886] = 293.81375
df_gas_mean_new_7['GAS MEAN TEMP'][13893] = 295.035
df_gas_mean_new_7['GAS MEAN TEMP'][13898:13901] = 295.217
df_gas_mean_new_7['GAS MEAN TEMP'][14916] = 270.82
df_gas_mean_new_7['GAS MEAN TEMP'][14945] = 270.72

### 일별로 데이터가 몇개 있는지 Count

#### 00시 ~ 24시 : 24 * 60 = 1440 
#### 09시 ~ 24시 : 15 * 60 = 900

In [5]:
df_gas_count_dict = { 1: [],
                2: [],
                3: [],
                4: [],
                5 : [],
                6 : [],
                7 : [],
                8 : []}
for idx in range(1,9):
    df_gas_count = globals()['df_gas_mean_new_' + str(idx)].loc[:,'date'].value_counts(sort = False)
    for j in range(1,len(df_gas_count)+1):
        df_gas_count_dict[idx].append(df_gas_count.iloc[j-1])

print(df_gas_count_dict)

{1: [900, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1439, 1440, 1440, 1440, 1440, 1438, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440], 2: [1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440], 3: [1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1439, 1439, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440], 4: [1440, 1440, 1440, 1440, 1440, 1440, 1438, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440], 5: [1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1438, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440], 6: [1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1438,

#### 1월 Data EDA

In [6]:
df_gas_mean_new_1.describe()

,date,GAS MEAN TEMP
count,4.409700e+04,44097.000000
mean,2.023012e+07,262.060428
std,8.842902e+00,61.242246
min,2.023010e+07,49.405500
25%,2.023011e+07,276.027000
50%,2.023012e+07,280.048000
75%,2.023012e+07,283.949500
max,2.023013e+07,313.038000


#### window size : 1시간 으로 slicing -> 간격 : 60, overlap : 10

In [7]:
mean_temp = []
#hyper parameter
DATA = df_gas_mean_new_1
LEN = len(DATA['GAS MEAN TEMP'])

# 몫과 나머지 구하기
share, remainder = (LEN // 60), (LEN % 60)
print("share, remainder : ", share, remainder)

for i in range(share):
    mean = round(DATA['GAS MEAN TEMP'][(i)*60:(i+1)*60].mean(),2) # (0,60),(60,120)
    mean_temp.append(mean)

if remainder != 0:
    mean_temp.append(DATA['GAS MEAN TEMP'][share * 60:].mean())

print("mean_temp length : ", len(mean_temp))


share, remainder :  734 57
mean_temp length :  735


In [8]:

plt.figure(figsize = (30,10))


<function matplotlib.pyplot.hist(x, bins=None, range=None, density=False, weights=None, cumulative=False, bottom=None, histtype='bar', align='mid', orientation='vertical', rwidth=None, log=False, color=None, label=None, stacked=False, *, data=None, **kwargs)>

<Figure size 3000x1000 with 0 Axes>

In [9]:
# for i in range(8):
#     pivot_data = pd.read_csv('./csv파일/2451_general_pivot_{}.csv'.format(i+1), encoding = 'cp949')
#     pkg_data = pd.read_csv('./csv파일/2451_general_pkg_list_{}.CSV'.format(i+1), encoding = 'cp949')
#     pivot_df = pd.DataFrame(pivot_data)
#     pkg_df = pd.DataFrame(pkg_data) 
#     one = pivot_df.loc[:, 'ds_date'].value_counts(sort=False)
#     two = pkg_df.loc[:, 'ds_date'].value_counts(sort=False)
#     result = pd.concat([one, two], axis =1 )
#     print(result)

In [10]:
# for i in range(8):
#     pivot_data = pd.read_csv('./csv파일/2451_general_pivot_{}.csv'.format(i+1), encoding = 'cp949')
    
#     pivot_df = pd.DataFrame(pivot_data)

#     one = pivot_df.loc[:, 'ds_date'].value_counts(sort=False)

#     print(one)

In [11]:
# for i in range(8):
#     pivot_data = pd.read_csv('./csv파일/2451_general_pkg_list_{}.csv'.format(i+1), encoding = 'cp949')
#     pkg_data = pd.read_csv('./한화오션_예지보전_데이터/2451_general_pkg_list_{}.csv'.format(i+1), encoding = 'cp949')
#     pivot_df = pd.DataFrame(pivot_data)
#     pkg_df = pd.DataFrame(pkg_data)
#     one = pivot_df.loc[:, 'ds_date'].value_counts(sort=False)
#     two = pkg_df.loc[:, 'ds_date'].value_counts(sort=False)
#     result = pd.concat([one, two], axis =1 )
#     print(result)

In [12]:
import seaborn as sns